In [1]:
import numpy as np
import pandas as pd

# 버전 문제로 생기는 warning 안뜨게 하는 법
import warnings
warnings.filterwarnings(action='ignore')

# 시간, 날짜 control 위한 패키지 설치
from datetime        import date, datetime, timedelta
from dateutil.parser import parse

import json
import urllib

# titanic dataset
import seaborn as sns

# 비정형 데이터 처리를 위한 라이브러리
from bs4 import BeautifulSoup
import requests
import re

print('numpy version - ', np.__version__)
print('pandas version - ', pd.__version__)

numpy version -  1.20.3
pandas version -  1.3.4


# 다중 인덱스
- 여러개의 계층을 갖는 인덱스
- DataFrame 만들 때 columns = [[1], [2]] 이런 구조로 하게 되는데, [1] 에서 상위(그룹) 인덱스 정의
- 하위 인덱스[2]와 개수를 맞춰서 하게 되면 해당 하위 인덱스들이 그룹[1]으로 묶이게 됨
- 사용의 편리성을 위해 열 인덱스의 이름을 부여해서 사용할 수 있음

### 다중 인덱스 (열)

In [6]:
multi_col_frm = pd.DataFrame( np.round(np.random.randn(6, 4), 2),
                              columns = [
                                  ['Grp01', 'Grp01', 'Grp02', 'Grp02'] , # 그룹으로 관리하는 index(상위 인덱스) - 하위 인덱스와 개수 맞춰서
                                  ['col02', 'col04', 'col03', 'col01']
                              ])

Grp01       Grp02      
  col02 col04 col03 col01
0  1.31  1.03  0.43  0.82
1 -0.40  1.72  0.19 -0.51
2  0.45  0.46  1.61  0.18
3 -0.14 -0.36  0.21  0.97
4 -0.22  0.95  0.57 -0.27
5 -0.48  0.50 -1.66 -0.89

In [8]:
# 인덱스 별로 각각의 이름을 부여해서 사용 가능
multi_col_frm.columns.names = ['GrpIdx', 'ColIdx']

GrpIdx Grp01       Grp02      
ColIdx col02 col04 col03 col01
0       1.31  1.03  0.43  0.82
1      -0.40  1.72  0.19 -0.51
2       0.45  0.46  1.61  0.18
3      -0.14 -0.36  0.21  0.97
4      -0.22  0.95  0.57 -0.27
5      -0.48  0.50 -1.66 -0.89

In [9]:
multi_col_frm.columns.names

FrozenList(['GrpIdx', 'ColIdx'])

### 다중 인덱스 (행)

In [11]:
multi_idx_frm = pd.DataFrame( np.round(np.random.randn(6, 4), 2),
                              columns = [
                                  ['Grp01', 'Grp01', 'Grp02', 'Grp02'] , # 그룹으로 관리하는 index(상위 인덱스) - 하위 인덱스와 개수 맞춰서
                                  ['col02', 'col04', 'col03', 'col01']
                              ],
                              index = [
                                  ['M', 'M', 'M', 'F', 'F', 'F'] ,              # 그룹이 되는 인덱스
                                  [ 'id_' + str(idx) for idx in range(6)]      # 우리가 아는 인덱스
                              ])
multi_idx_frm.columns.names = ['GrpIdx', 'ColIdx']  # 열 인덱스 이름 부여
multi_idx_frm.index.names = ['GenderIdx', 'RowIdx'] # 행 인덱스 이름 부여
multi_idx_frm

GrpIdx           Grp01       Grp02      
ColIdx           col02 col04 col03 col01
GenderIdx RowIdx                        
M         id_0    1.06 -1.16 -0.38  0.37
          id_1    2.22  0.07 -0.46 -1.45
          id_2    1.13  0.15  1.01  1.08
F         id_3    0.62 -1.30 -0.03 -0.13
          id_4   -1.38  1.56  1.53  1.90
          id_5    1.76  0.73 -1.39  2.10

### 다중 인덱스 함수
- stack() : 열 인덱스 > 행 인덱스로 바꿔줌
  - DataFrame 이 반시계 방향으로 90도 회전한 모습
- unstack() : 행 인덱스 > 열 인덱스로 바꿔줌
  - DataFrame 이 시계 방향으로 90도 회전한 모습

> 둘다 열<->행이 transpose 된게 아니라 열 인덱스가 행 인덱스의 가장 하위 인덱스로 위치하게 되고(stack), 행 인덱스가 열 인덱스의 가장 하위 인덱스로 위치하게 되는 형태(unstack)

In [13]:
multi_idx_frm.stack()

GrpIdx                   Grp01  Grp02
GenderIdx RowIdx ColIdx              
M         id_0   col01     NaN   0.37
                 col02    1.06    NaN
                 col03     NaN  -0.38
                 col04   -1.16    NaN
          id_1   col01     NaN  -1.45
                 col02    2.22    NaN
                 col03     NaN  -0.46
                 col04    0.07    NaN
          id_2   col01     NaN   1.08
                 col02    1.13    NaN
                 col03     NaN   1.01
                 col04    0.15    NaN
F         id_3   col01     NaN  -0.13
                 col02    0.62    NaN
                 col03     NaN  -0.03
                 col04   -1.30    NaN
          id_4   col01     NaN   1.90
                 col02   -1.38    NaN
                 col03     NaN   1.53
                 col04    1.56    NaN
          id_5   col01     NaN   2.10
                 col02    1.76    NaN
                 col03     NaN  -1.39
                 col04    0.73    NaN

In [14]:
multi_idx_frm.unstack()

GrpIdx    Grp01                                                       ...  \
ColIdx    col02                               col04                   ...   
RowIdx     id_0  id_1  id_2  id_3  id_4  id_5  id_0  id_1  id_2 id_3  ...   
GenderIdx                                                             ...   
F           NaN   NaN   NaN  0.62 -1.38  1.76   NaN   NaN   NaN -1.3  ...   
M          1.06  2.22  1.13   NaN   NaN   NaN -1.16  0.07  0.15  NaN  ...   

GrpIdx    Grp02                                                      
ColIdx    col03                   col01                              
RowIdx     id_2  id_3  id_4  id_5  id_0  id_1  id_2  id_3 id_4 id_5  
GenderIdx                                                            
F           NaN -0.03  1.53 -1.39   NaN   NaN   NaN -0.13  1.9  2.1  
M          1.01   NaN   NaN   NaN  0.37 -1.45  1.08   NaN  NaN  NaN  

[2 rows x 24 columns]